### [4] 第四部分：TensorFlow 模型导出  <br/>
   &emsp; 4.1 使用 SavedModel 完整导出模型 <br/>
   &emsp; 4.2 Keras 自有的模型导出格式（Jinpeng） <br/>

In [21]:
import tensorflow as tf
from tensorflow_core.examples.tutorials.mnist import input_data
import numpy as np
class MNISTLoader_my_download():
    def __init__(self):
        # 读取数据，预先已经下载了相应的数据直
        mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
        self.train_data = mnist.train.images
        self.train_label = mnist.train.labels
        self.test_data = mnist.test.images
        self.test_label = mnist.test.labels
        
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 784, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 784, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
        
    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]


In [22]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)  # 第一层神经元的个数为100
        self.dense2 = tf.keras.layers.Dense(units=10)   # 第二层神经元的个数为10,输出一个样本的维度为10

    @tf.function
    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

In [23]:
# 定义一些模型超参数：
num_epochs = 5
batch_size = 50
learning_rate = 0.001

# 实例化模型和数据读取类，并实例化一个 tf.keras.optimizer 的优化器（这里使用常用的 Adam 优化器）：
model = MLP()
# data_loader = MNISTLoader() # 导入数据 
data_loader = MNISTLoader_my_download()  # 导入数据
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # 更新梯度

# num_batches = int(mnist.train.num_examples // batch_size * num_epochs)
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
batch 0: loss 2.302643
batch 1: loss 2.301799
batch 2: loss 2.302603
batch 3: loss 2.302225
batch 4: loss 2.301180
batch 5: loss 2.301542
batch 6: loss 2.300638
batch 7: loss 2.299495
batch 8: loss 2.299714
batch 9: loss 2.298243
batch 10: loss 2.301099
batch 11: loss 2.300594
batch 12: loss 2.299020
batch 13: loss 2.299688
batch 14: loss 2.297707
batch 15: loss 2.296647
batch 16: loss 2.299434
batch 17: loss 2.297530
batch 18: loss 2.297919
batch 19: loss 2.299123
batch 20: loss 2.297805
batch 21: loss 2.296417
batch 22: loss 2.293397
batch 23: loss 2.296087
batch 24: loss 2.296263
batch 25: loss 2.290941
batch 26: loss 2.294185
batch 27: loss 2.292846
batch 28: loss 2.290687
batch 29: loss 2.293303
batch 30: loss 2.291676
batch 31: loss 2.293294
batch 32: loss 2.291065
batch 33: loss 2.290572


batch 351: loss 1.734943
batch 352: loss 1.717589
batch 353: loss 1.746754
batch 354: loss 1.817168
batch 355: loss 1.747950
batch 356: loss 1.735922
batch 357: loss 1.736252
batch 358: loss 1.773902
batch 359: loss 1.662208
batch 360: loss 1.742712
batch 361: loss 1.639705
batch 362: loss 1.681749
batch 363: loss 1.602814
batch 364: loss 1.688803
batch 365: loss 1.750672
batch 366: loss 1.747290
batch 367: loss 1.714188
batch 368: loss 1.724599
batch 369: loss 1.717234
batch 370: loss 1.673671
batch 371: loss 1.728030
batch 372: loss 1.749232
batch 373: loss 1.645709
batch 374: loss 1.740792
batch 375: loss 1.592468
batch 376: loss 1.730510
batch 377: loss 1.787460
batch 378: loss 1.676455
batch 379: loss 1.675340
batch 380: loss 1.606389
batch 381: loss 1.692972
batch 382: loss 1.683313
batch 383: loss 1.647904
batch 384: loss 1.715754
batch 385: loss 1.656535
batch 386: loss 1.679291
batch 387: loss 1.630689
batch 388: loss 1.787637
batch 389: loss 1.683315
batch 390: loss 1.655793


batch 690: loss 1.266107
batch 691: loss 1.210415
batch 692: loss 1.088680
batch 693: loss 1.138302
batch 694: loss 1.195412
batch 695: loss 1.119628
batch 696: loss 1.118680
batch 697: loss 1.020716
batch 698: loss 1.160108
batch 699: loss 1.108284
batch 700: loss 1.083138
batch 701: loss 1.135447
batch 702: loss 1.051618
batch 703: loss 1.057718
batch 704: loss 1.091801
batch 705: loss 1.152697
batch 706: loss 1.246926
batch 707: loss 1.114609
batch 708: loss 1.164019
batch 709: loss 1.260595
batch 710: loss 1.030509
batch 711: loss 0.989021
batch 712: loss 0.983583
batch 713: loss 1.108461
batch 714: loss 1.115162
batch 715: loss 1.181884
batch 716: loss 1.038350
batch 717: loss 1.034064
batch 718: loss 1.134526
batch 719: loss 1.259911
batch 720: loss 1.068229
batch 721: loss 1.036055
batch 722: loss 1.078259
batch 723: loss 1.103709
batch 724: loss 1.129956
batch 725: loss 1.096777
batch 726: loss 1.066424
batch 727: loss 1.131852
batch 728: loss 1.077152
batch 729: loss 1.117536


batch 1045: loss 0.734064
batch 1046: loss 0.897324
batch 1047: loss 0.813515
batch 1048: loss 1.018422
batch 1049: loss 0.777984
batch 1050: loss 0.857251
batch 1051: loss 0.931520
batch 1052: loss 0.946347
batch 1053: loss 0.892197
batch 1054: loss 0.809341
batch 1055: loss 0.723361
batch 1056: loss 0.817453
batch 1057: loss 0.782016
batch 1058: loss 0.831183
batch 1059: loss 0.949572
batch 1060: loss 0.842253
batch 1061: loss 0.873905
batch 1062: loss 0.817738
batch 1063: loss 0.835279
batch 1064: loss 0.809385
batch 1065: loss 0.795710
batch 1066: loss 0.877624
batch 1067: loss 0.843203
batch 1068: loss 0.930756
batch 1069: loss 0.852211
batch 1070: loss 0.881119
batch 1071: loss 0.906188
batch 1072: loss 0.707146
batch 1073: loss 0.815194
batch 1074: loss 0.954605
batch 1075: loss 0.990198
batch 1076: loss 0.833570
batch 1077: loss 0.868966
batch 1078: loss 0.732350
batch 1079: loss 0.783846
batch 1080: loss 0.881832
batch 1081: loss 0.884563
batch 1082: loss 0.714804
batch 1083: 

batch 1363: loss 0.696965
batch 1364: loss 0.732575
batch 1365: loss 0.668751
batch 1366: loss 0.669507
batch 1367: loss 0.590747
batch 1368: loss 0.621324
batch 1369: loss 0.618797
batch 1370: loss 0.773376
batch 1371: loss 0.568595
batch 1372: loss 0.590521
batch 1373: loss 0.636020
batch 1374: loss 0.768477
batch 1375: loss 0.810796
batch 1376: loss 0.916021
batch 1377: loss 0.719857
batch 1378: loss 0.614954
batch 1379: loss 0.757353
batch 1380: loss 0.685695
batch 1381: loss 0.685879
batch 1382: loss 0.644755
batch 1383: loss 0.821842
batch 1384: loss 0.685386
batch 1385: loss 0.792516
batch 1386: loss 0.697359
batch 1387: loss 0.669218
batch 1388: loss 0.747285
batch 1389: loss 0.683494
batch 1390: loss 0.642349
batch 1391: loss 0.671826
batch 1392: loss 0.730739
batch 1393: loss 0.634707
batch 1394: loss 0.567053
batch 1395: loss 0.839080
batch 1396: loss 0.729280
batch 1397: loss 0.839153
batch 1398: loss 0.554817
batch 1399: loss 0.895457
batch 1400: loss 0.738119
batch 1401: 

...

batch 3928: loss 0.417981
batch 3929: loss 0.323852
batch 3930: loss 0.318418
batch 3931: loss 0.346079
batch 3932: loss 0.480213
batch 3933: loss 0.423452
batch 3934: loss 0.237804
batch 3935: loss 0.189027
batch 3936: loss 0.584672
batch 3937: loss 0.301954
batch 3938: loss 0.319973
batch 3939: loss 0.352234
batch 3940: loss 0.348416
batch 3941: loss 0.461860
batch 3942: loss 0.334555
batch 3943: loss 0.295372
batch 3944: loss 0.359757
batch 3945: loss 0.339381
batch 3946: loss 0.498045
batch 3947: loss 0.505108
batch 3948: loss 0.282826
batch 3949: loss 0.258680
batch 3950: loss 0.336807
batch 3951: loss 0.609317
batch 3952: loss 0.371570
batch 3953: loss 0.603882
batch 3954: loss 0.546476
batch 3955: loss 0.224182
batch 3956: loss 0.347859
batch 3957: loss 0.264474
batch 3958: loss 0.325716
batch 3959: loss 0.358475
batch 3960: loss 0.518516
batch 3961: loss 0.273193
batch 3962: loss 0.533688
batch 3963: loss 0.244999
batch 3964: loss 0.346748
batch 3965: loss 0.406168
batch 3966: 

batch 4245: loss 0.405179
batch 4246: loss 0.245954
batch 4247: loss 0.391496
batch 4248: loss 0.462544
batch 4249: loss 0.480049
batch 4250: loss 0.360296
batch 4251: loss 0.593154
batch 4252: loss 0.290058
batch 4253: loss 0.370973
batch 4254: loss 0.447295
batch 4255: loss 0.220289
batch 4256: loss 0.276855
batch 4257: loss 0.373578
batch 4258: loss 0.473255
batch 4259: loss 0.334427
batch 4260: loss 0.609165
batch 4261: loss 0.443520
batch 4262: loss 0.222823
batch 4263: loss 0.306586
batch 4264: loss 0.524174
batch 4265: loss 0.545110
batch 4266: loss 0.228805
batch 4267: loss 0.403489
batch 4268: loss 0.245593
batch 4269: loss 0.304942
batch 4270: loss 0.496464
batch 4271: loss 0.338786
batch 4272: loss 0.245557
batch 4273: loss 0.204008
batch 4274: loss 0.317925
batch 4275: loss 0.247081
batch 4276: loss 0.501180
batch 4277: loss 0.692021
batch 4278: loss 0.335666
batch 4279: loss 0.383366
batch 4280: loss 0.561169
batch 4281: loss 0.372173
batch 4282: loss 0.260375
batch 4283: 

batch 4565: loss 0.393465
batch 4566: loss 0.458079
batch 4567: loss 0.469266
batch 4568: loss 0.529657
batch 4569: loss 0.340997
batch 4570: loss 0.503533
batch 4571: loss 0.178522
batch 4572: loss 0.226068
batch 4573: loss 0.328688
batch 4574: loss 0.463463
batch 4575: loss 0.375662
batch 4576: loss 0.307602
batch 4577: loss 0.474629
batch 4578: loss 0.278554
batch 4579: loss 0.283665
batch 4580: loss 0.383428
batch 4581: loss 0.333654
batch 4582: loss 0.394892
batch 4583: loss 0.512433
batch 4584: loss 0.295264
batch 4585: loss 0.549163
batch 4586: loss 0.273746
batch 4587: loss 0.474226
batch 4588: loss 0.272300
batch 4589: loss 0.212039
batch 4590: loss 0.260825
batch 4591: loss 0.358859
batch 4592: loss 0.477550
batch 4593: loss 0.339917
batch 4594: loss 0.343811
batch 4595: loss 0.295315
batch 4596: loss 0.518301
batch 4597: loss 0.332548
batch 4598: loss 0.251485
batch 4599: loss 0.288740
batch 4600: loss 0.211657
batch 4601: loss 0.358484
batch 4602: loss 0.373101
batch 4603: 

batch 4908: loss 0.339196
batch 4909: loss 0.327297
batch 4910: loss 0.703279
batch 4911: loss 0.526917
batch 4912: loss 0.280244
batch 4913: loss 0.403481
batch 4914: loss 0.308475
batch 4915: loss 0.422574
batch 4916: loss 0.402488
batch 4917: loss 0.228908
batch 4918: loss 0.304811
batch 4919: loss 0.384238
batch 4920: loss 0.296883
batch 4921: loss 0.431715
batch 4922: loss 0.370844
batch 4923: loss 0.335986
batch 4924: loss 0.272550
batch 4925: loss 0.297565
batch 4926: loss 0.201336
batch 4927: loss 0.454298
batch 4928: loss 0.445588
batch 4929: loss 0.186247
batch 4930: loss 0.556872
batch 4931: loss 0.727795
batch 4932: loss 0.340169
batch 4933: loss 0.272626
batch 4934: loss 0.527880
batch 4935: loss 0.279652
batch 4936: loss 0.240741
batch 4937: loss 0.380119
batch 4938: loss 0.151014
batch 4939: loss 0.413697
batch 4940: loss 0.361606
batch 4941: loss 0.382870
batch 4942: loss 0.203020
batch 4943: loss 0.479180
batch 4944: loss 0.403101
batch 4945: loss 0.546179
batch 4946: 

batch 5225: loss 0.248728
batch 5226: loss 0.460969
batch 5227: loss 0.206974
batch 5228: loss 0.359044
batch 5229: loss 0.365082
batch 5230: loss 0.583996
batch 5231: loss 0.350794
batch 5232: loss 0.329026
batch 5233: loss 0.353356
batch 5234: loss 0.275648
batch 5235: loss 0.164922
batch 5236: loss 0.431761
batch 5237: loss 0.327112
batch 5238: loss 0.240511
batch 5239: loss 0.307412
batch 5240: loss 0.418059
batch 5241: loss 0.532717
batch 5242: loss 0.408275
batch 5243: loss 0.458353
batch 5244: loss 0.457882
batch 5245: loss 0.241416
batch 5246: loss 0.338607
batch 5247: loss 0.309174
batch 5248: loss 0.360754
batch 5249: loss 0.294749
batch 5250: loss 0.262626
batch 5251: loss 0.146086
batch 5252: loss 0.457767
batch 5253: loss 0.446854
batch 5254: loss 0.224809
batch 5255: loss 0.211662
batch 5256: loss 0.294999
batch 5257: loss 0.145654
batch 5258: loss 0.399267
batch 5259: loss 0.214471
batch 5260: loss 0.361466
batch 5261: loss 0.614813
batch 5262: loss 0.327603
batch 5263: 

In [27]:
tf.saved_model.save(model, "saved\\2")

INFO:tensorflow:Assets written to: saved\2\assets


In [29]:
batch_size = 50
model = tf.saved_model.load("saved\\2")

categorical_accuracy = tf.keras.metrics.CategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.call(data_loader.test_data[start_index: end_index])
    categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % categorical_accuracy.result())

test accuracy: 0.906900


## 4.2 Keras 自有的模型导出格式（Jinpeng）

由于历史原因，我们在有些场景也会用到 Keras 的 Sequential 和 Functional 模式的自有模型导出格式（H5）。我们以 keras 模型训练和保存为例进行讲解，如下是 keras 官方的 mnist 模型训练样例。